In [39]:
import sys
sys.path.append("/Users/adyaabba/GSOC_2025") 


In [40]:
import importlib.util
spec = importlib.util.find_spec("moha")
print(spec)


ModuleSpec(name='moha', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7fda59403460>, origin='/Users/adyaabba/GSOC_2025/moha/__init__.py', submodule_search_locations=['/Users/adyaabba/GSOC_2025/moha'])


In [45]:
r"""Model Hamiltonian classes."""

import numpy as np

from scipy.sparse import csr_matrix, diags, lil_matrix, hstack, vstack, \
    SparseEfficiencyWarning

from moha.api import HamiltonianAPI

from moha.utils import convert_indices, expand_sym

from typing import Union

from moha.rauk.rauk import assign_rauk_parameters
from moha.rauk.PariserParr import compute_overlap
import warnings

warnings.simplefilter('ignore',
                      SparseEfficiencyWarning)

__all__ = [
    "HamPPP",
    "HamHuck",
    "HamHub",
    "HamHeisenberg"
]


class HamPPP(HamiltonianAPI):
    r"""Pariser-Parr-Pople Hamiltonian."""

    def __init__(
            self,
            connectivity: Union[list, np.ndarray],
            alpha=-0.414,
            beta=-0.0533,
            u_onsite=None,
            gamma=None,
            charges=None,
            sym=1,
            atom_dictionary=None,
            bond_dictionary=None,
            orbital_overlap=None
    ):
        r"""
        Initialize Pariser-Parr-Pople Hamiltonian.

        Parameters
        ----------
        connectivity: list, np.ndarray
            list of tuples that specifies sites and bonds between them
            or symmetric np.ndarray of shape (n_sites, n_sites) that specifies
            the connectivity between sites.
            For example, for a linear chain of 4 sites, the connectivity
            can be specified as [(C1, C2, 1), (C2, C3, 1), (C3, C4, 1)]
        alpha: float
            specifies the site energy if all sites are equivalent.
            Default value is the 2p-pi orbital of Carbon
        beta: float
            specifies the resonance energy, hopping term,
            if all bonds are equivalent.
            The default value is appropriate for a pi-bond between Carbon atoms
        u_onsite: np.ndarray
            on-site Coulomb interaction; 1d np.ndarray
        gamma: np.ndarray
            parameter that specifies long-range Coulomb interaction; 2d
        charges: np.ndarray
            Charges on sites; 1d np.ndarray
        sym: int
             symmetry of the Hamiltonian: int [1, 2, 4, 8]. Default is 1

        Notes
        -----
        The Hamiltonian is given by:

        .. math::
            \begin{align}
            \hat{H}_{\mathrm{PPP}\mathrm{P}} &=
            \sum_{pq}h_{pq} a_{p}^{\dagger}a_{q} \\
            &+ \sum_{p} U_{p} \hat{n}_{p \alpha}
            \hat{n}_{p\beta} \\
            &+ \frac{1}{2}\sum_{p\neq q}\gamma_{pq}\left(\hat{n}_{p\alpha}
             + \hat{n}_{p \beta}-Q_{p}\right)
            \left(\hat{n}_{q \alpha}+\hat{n}_{q\beta}-Q_{q}\right)
            \end{align}

        """
        self._sym = sym
        self.n_sites = None
        self.connectivity = connectivity
        self.alpha = alpha
        self.beta = beta
        self.u_onsite = u_onsite
        self.gamma = gamma
        self.charges = charges
        self.atom_types = None
        self.atoms_dist = None
        self.atoms_num, self.connectivity_matrix = \
            self.generate_connectivity_matrix()
        self.zero_energy = None
        self.one_body = None
        self.two_body = None
        self.bond_dictionary = bond_dictionary
        self.atom_dictionary = atom_dictionary
        self.orbital_overlap = orbital_overlap


In [46]:
from moha.api import HamiltonianAPI

class HamHeisenberg(HamiltonianAPI):
    r"""XXZ Heisenberg Hamiltonian."""

    def __init__(self,
                 mu: np.ndarray,
                 J_eq: np.ndarray,
                 J_ax: np.ndarray,
                 connectivity: np.ndarray = None
                 ):
        r"""Initialize XXZ Heisenberg Hamiltonian.

        Parameters
        ----------
        mu: np.ndarray
            Zeeman term
        J_eq: np.ndarray
            J equatorial term
        J_ax: np.ndarray
            J axial term
        connectivity: np.ndarray
            symmetric numpy array that specifies the connectivity between sites

        Notes
        -----
        The form of the Hamiltonian is given by:

        .. math::
            \hat{H}_{X X Z}=\sum_p\left(\mu_p^Z-J_{p p}^{\mathrm{eq}}\right)
            S_p^Z+\sum_{p q} J_{p q}^{\mathrm{ax}} S_p^Z S_q^Z+\sum_{p q}
            J_{p q}^{\mathrm{eq}} S_p^{+} S_q^{-}

        """
        if connectivity is not None:
            self.connectivity = connectivity
            self.n_sites = connectivity.shape[0]
            # if J_eq and J_ax are floats then convert them to numpy arrays
            # by multiplying with connectivity matrix
            if isinstance(J_eq, (int, float)):
                self.J_eq = J_eq * connectivity
                self.J_ax = J_ax * connectivity
                self.mu = mu * np.ones(self.n_sites)
            else:
                raise TypeError("Connectivity matrix is provided, "
                                "J_eq, J_ax, and mu should be floats")
        else:
            if isinstance(J_eq, np.ndarray) and \
               isinstance(J_ax, np.ndarray) and \
               isinstance(mu, np.ndarray) and \
               J_eq.shape == J_ax.shape and \
               mu.shape[0] == J_eq.shape[0]:
                self.n_sites = J_eq.shape[0]
                self.J_eq = J_eq
                self.J_ax = J_ax
            else:
                raise TypeError("J_eq and J_ax should be numpy arrays of the same shape")  # noqa: E501

        self.mu = np.array(mu)
        self.atom_types = None
        self.zero_energy = None
        self.one_body = None
        self.two_body = None
        self._sym = 1

In [49]:
class HamTJ(HamPPP, HamHeisenberg):
    r"""t-J Hamiltonian."""

    def __init__(self,
                 connectivity_ppp: Union[list, np.ndarray],
                 connectivity_heisenberg: np.ndarray,
                 connectivity: Union[list, np.ndarray],
                 alpha=-0.414,
                 beta=-0.0533,
                 u_onsite=0,
                 gamma=0,
                 charges=None,
                 sym=1,
                 atom_dictionary=None,
                 bond_dictionary=None,
                 orbital_overlap=None,
                 affinity_dct=None,
                 Rxy_list=None,
                 mu=None,
                 J_eq=None,
                 J_ax=None
                 ):
        """
        Initialize the t-J Hamiltonian

        Parameters:
        - connectivity_ppp: Connectivity matrix for PPP part
        - connectivity_heisenberg: Connectivity matrix for Heisenberg part
        - connectivity: Combined connectivity matrix
        - alpha: t-J interaction parameter
        - beta: J-U interaction parameter
        - u_onsite: On-site U potential
        - charges: Array of charges (default to array of ones)
        - J_eq: Heisenberg exchange interaction parameter
        - J_ax: Heisenberg axial interaction parameter
        - affinity_dct: Dictionary of affinities (optional)
        - Rxy_list: List of Rxy for calculating interactions (optional)
        """
        # Default charges to an array of ones if not provided
        if charges is None:
            charges = np.ones(len(connectivity_ppp))

        mu = np.zeros(connectivity_heisenberg.shape[0])
        
        # Initialize the PPP part
        self.ocupation_part = HamPPP(connectivity=connectivity,
                                      alpha=alpha,
                                      beta=beta,
                                      u_onsite=u_onsite,
                                      affinity_dct=affinity_dct,
                                      Rxy_list=Rxy_list)

        connectivity_matrix = np.asarray(self.ocupation_part.connectivity_matrix.todense())

        # Initialize the Heisenberg part
        self.spin_part = HamHeisenberg(mu=mu,
                                       J_eq=J_eq,
                                       J_ax=J_ax,
                                       connectivity=connectivity_matrix)
        
    def generate_zero_body_integral(self):
        r"""Generate zero body integral.
            Returns
            -------
            float
        """
        self.zero_energy = self.ocupation_part.generate_zero_body_integral(
        ) + self.spin_part.generate_zero_body_integral()
        return self.zero_energy

    def generate_one_body_integral(self, basis: str, dense: bool):
        r"""
            Generate one body integral in spatial or spin orbital basis.
            Parameters
            ----------
            basis: str
                ['spatial', 'spin orbital']
            dense: bool
                Dense or sparse matrix; default False
            Returns
            -------
            scipy.sparse.csr_matrix or np.ndarray
        """
        one_body_ppp = self.ocupation_part.generate_one_body_integral(
            basis, dense)
        one_body_heisenberg = self.spin_part.generate_one_body_integral(
            dense, basis)
        self.one_body = one_body_ppp + one_body_heisenberg
        return self.one_body

    def generate_two_body_integral(self, basis: str, dense: bool, sym=1):
        r"""
            Generate two body integral in spatial or spin orbital basis.
            Parameters
            ----------
            basis: str
                ['spatial', 'spin orbital']
            dense: bool
                Dense or sparse matrix; default False
            sym: int
                Symmetry -- [2, 4, 8] default is 1
            Returns
            -------
            scipy.sparse.csr_matrix or np.ndarray
        """
        two_body_ppp = self.ocupation_part.generate_two_body_integral(
            basis, dense, sym)
        two_body_heisenberg = self.spin_part.generate_two_body_integral(
            sym, dense, basis)
        self.two_body = two_body_ppp + two_body_heisenberg
        return self.two_body
